In [ ]:
%session_id_prefix delta-dataframe-
%glue_version 3.0
%idle_timeout 60
%connections <your-delta-connection-name>
%%configure
{
  "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
  "--extra-py-files": "/tmp/etl-delta-core_2.12-1.0.0.jar" # for custom connector
  # "--extra-py-files": "/tmp/delta-core_2.12-1.0.0.jar" # for marketplace connector
}

In [ ]:
bucket_name = "<Your S3 bucket name>"
bucket_prefix = "<Your S3 bucket prefix>"
database_name = "delta_dataframe"
table_name = "products"
table_prefix = f"{bucket_prefix}/{database_name}/{table_name}"
table_location = f"s3://{bucket_name}/{table_prefix}"

## Initialize SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark.stop()
spark = SparkSession.builder.getOrCreate()

## Clean up existing resources

In [ ]:
import boto3

## Delete files in S3
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.filter(Prefix=f"{table_prefix}/").delete()

## Drop tables in Glue Data Catalog
try:
    glue = boto3.client('glue')
    glue.delete_table(DatabaseName=database_name, Name=table_name)
except glue.exceptions.EntityNotFoundException:
    print(f"{database_name}.{table_name} does not exist")
try:
    glue = boto3.client('glue')
    glue.delete_table(DatabaseName=database_name, Name='testTable')
except glue.exceptions.EntityNotFoundException:
    print(f"{database_name}.testTable does not exist")


## Create Delta table with sample data

In [ ]:
from pyspark.sql import Row
import time

ut = time.time()

product = [
    {'product_id': '00001', 'product_name': 'Heater', 'price': 250, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00002', 'product_name': 'Thermostat', 'price': 400, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00003', 'product_name': 'Television', 'price': 600, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00004', 'product_name': 'Blender', 'price': 100, 'category': 'Electronics', 'updated_at': ut},
    {'product_id': '00005', 'product_name': 'USB charger', 'price': 50, 'category': 'Electronics', 'updated_at': ut}
]

df_products = spark.createDataFrame(Row(**x) for x in product)

In [ ]:
# Create table in the metastore using DataFrame's schema and write data to it
df_products.write.format("delta").mode("overwrite").option("path",table_location).saveAsTable(f"{database_name}.{table_name}")

In [ ]:
# Create a Deltatable with path (not in metastore) using DataFrame's schema and write/overwrite data to it
df_products.write.format("delta").mode("overwrite").save(table_location)

In [ ]:
# Create a Deltatable using the DeltaTableBuilder API using a dataframe's schema
from delta.tables import DeltaTable
deltaTable = DeltaTable.create(spark).tableName(f"{database_name}.testTable").addColumns(df_products.schema).location(table_location+"_testTable").execute()

## Read from Delta table via DataFrame

In [ ]:
# query table in the metastore
df_products_read = spark.table(f"{database_name}.{table_name}")
df_products_read.show()



In [ ]:
# query table by path
df_products_read = spark.read. \
    format("delta"). \
    load(table_location)
df_products_read.show()

## Read from Delta table via DeltaLake library

In [ ]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, table_location) #query table from path
deltaTable.toDF().show()

In [ ]:
deltaTable = DeltaTable.forName(spark,f"{database_name}.{table_name}") #query table from metastore
deltaTable.toDF().show()

In [ ]:
deltaTabletest = DeltaTable.forName(spark,f"{database_name}.testTable").toDF().show()

## Modify schema/properties of DeltaLake table

In [ ]:
from pyspark.sql.functions import lit

df_products.withColumn("Currency",lit("USD")).write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("path", table_location) \
    .saveAsTable(f"{database_name}.{table_name}") # External table

## Insert records

In [ ]:
ut = time.time()
product_insert = [
    {'product_id': '00006', 'product_name': 'Book', 'price': 500, 'category': 'Stationery','Currency': 'INR', 'updated_at': ut}, # Insert
    {'product_id': '00007', 'product_name': 'Pen', 'price': 50, 'category': 'Stationery','Currency': 'USD', 'updated_at': ut} # Insert
]
df_product_insert = spark.createDataFrame(Row(**x) for x in product_insert)

df_product_insert.write.format("delta").mode("append").saveAsTable(f"{database_name}.{table_name}")

## Update records

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forName(spark, f"{database_name}.{table_name}")

# Declare the predicate by using a SQL-formatted string.
deltaTable.update(
    condition = "product_id = '00006'",
    set = { "Currency": "'USD'" }
)


## Delete records

In [ ]:
# delete product_id 00007

deltaTable = DeltaTable.forName(spark, f"{database_name}.{table_name}")
deltaTable.delete("product_id = '00007'")

## Upsert records into Delta table

In [ ]:
ut = time.time()

product_updates = [
    {'product_id': '00001', 'product_name': 'Heater', 'price': 400, 'category': 'Electronics', 'Currency': 'INR','updated_at': ut}, # Update
    {'product_id': '00007', 'product_name': 'Chair', 'price': 50, 'category': 'Furniture', 'Currency': 'INR','updated_at': ut} # Insert
]
df_product_updates = spark.createDataFrame(Row(**x) for x in product_updates)

deltaTable.alias("products").merge(
    df_product_updates.alias("updates"),
    "products.product_id = updates.product_id") \
    .whenMatchedUpdate(set = {
        "product_name": "updates.product_name",
        "price": "updates.price",
        "category": "updates.category",
        "updated_at": "updates.updated_at",
        "Currency": "updates.Currency"
    } ) \
    .whenNotMatchedInsert(values = {
        "product_id": "updates.product_id",
        "product_name": "updates.product_name",
        "price": "updates.price",
        "category": "updates.category",
        "updated_at": "updates.updated_at",
        "Currency": "updates.Currency"
    }
) \
.execute()
deltaTable.toDF().show()

## View History

In [ ]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, table_location)

fullHistoryDF = deltaTable.history() 
fullHistoryDF.show()

## Query with time travel

In [ ]:
df1 = spark.read.format("delta").option("timestampAsOf", "2022-04-28 12:44:07").load(table_location) #timestamp in YYYY-MM-DD HH:MM:SS
df2 = spark.read.format("delta").option("versionAsOf", 3).load(table_location)
df1.show()
df2.show()


# Roll Back

In [ ]:
#Restore to version 2 using dataframe

spark.read.format("delta") \
    .option("versionAsOf", 2) \
    .load(table_location) \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .save(table_location)


## Stop Session

In [ ]:
%stop_session